In [60]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, Aer, execute
from qiskit.aqua.components.optimizers import GSLS, L_BFGS_B, SLSQP, TNC, CRS, DIRECT_L, DIRECT_L_RAND, ESCH, ISRES
import math
import random
import numpy as np
from numpy import pi
import time
import itertools
#import matplotlib.pyplot as plt
import pickle
import enum
from typing import List, Tuple

In [61]:
# Info used to do calculations in parallel

clones = 12 # Number of parallel calculators (VMs)
clon = 0    # VM Calculator Number
file1 = '20210402_C1_A1_2nd_Circuit_Clon00'
file2 = '20210402_C1_A2_2nd_Circuit_Clon00'
file3 = '20210402_C1_A3_2nd_Circuit_Clon00'
file4 = '20210402_C1_A4_2nd_Circuit_Clon00'
file5 = '20210402_C1_A5_2nd_Circuit_Clon00'


In [62]:
class GuessStrategy(enum.Enum):
    one_bit_same_as_measured = 1
    two_bit_base = 2
    two_bit_neural_network = 3

## Auxiliary functions

In [63]:
""" save and load results to and from a file """
def save_results_to_disk(obj, name ):
    with open('results/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_results_from_file(name ):
    with open('results/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [64]:
def prepare_initial_state(theta, phase):
    """ Prepare initial state """
    return (0, math.cos(theta),
        math.sin(theta), 0)

In [65]:
def convert_counts_to_final_result(counts) -> int:
    """ Convert the execution result to the final measured value: 0 or 1 """
    if counts == "00":
        return 1
    if counts == "01":
        return 0
    if counts == "10" or counts == "11":
        return random.choice([0,1])
    return 1

In [66]:
def set_random_lambda(lambda0, lambda1):
    """ return a random choice from lambda0 and lambda1 with the correspondent key name """
    lambda_value = random.choice([lambda0, lambda1])
    if lambda_value == lambda0:
        return lambda_value, 0
    return lambda_value, 1

In [67]:
def compute_damping_channel(theta, phase, one_eta, phi_rx, phi_ry, backend) -> int:
    """ one-time execution of the amplitude damping circuit using the passed parameters 
        Returns: the execution measured result: either 0 or 1
    """
    qreg_q = QuantumRegister(3, 'q')
    creg_c = ClassicalRegister(2, 'c')

    initial_state = prepare_initial_state(theta, 0)

    circuit = QuantumCircuit(qreg_q, creg_c)
    circuit.initialize(initial_state, [0, 1])
    circuit.reset(qreg_q[2])
    circuit.cry(2 * one_eta, qreg_q[1], qreg_q[2])
    circuit.cx(qreg_q[2], qreg_q[1])
    circuit.rx(phi_rx, qreg_q[1])
    circuit.ry(phi_ry, qreg_q[1])
    circuit.barrier() 
    circuit.measure([0, 1], creg_c)

    counts = execute(circuit, backend, shots=1).result().get_counts(circuit)
    return convert_counts_to_final_result(counts)

In [68]:
def compute_random_damping_channel(configuration, backend):
    """ run a one-time execution on the given backend device randomly selectiong the lambda to be used and return the measured result """
    lambda_value, lambda_used = set_random_lambda(configuration['lambda0'], configuration['lambda1'])
    result = compute_damping_channel(configuration['theta'], 0, lambda_value, configuration['phi_rx'], configuration['phi_ry'], backend)

    return {
        "result": result,
        "lambda_used": lambda_used
    }

In [69]:
def guess_lambda_used(real_measured_result):
    """ Decides which lambda was used on the real execution.
        It is a silly guess.
        It returns the same lambda used as the measured result
    """
    return real_measured_result

In [70]:
def check_value(real_lambda, guess_lambda): 
    if real_lambda == guess_lambda: return 1
    return 0

In [71]:
def play_and_guess_one_case(configuration, backend):
    """ Execute a real execution with a random lambda from the two passed, 
        guess which one was used on the exection and
        check the result.
        Returns 1 on success (it was a correct guess) or 0 on fail (it was an incorrect guess) 
    """
    real_result = compute_random_damping_channel(configuration, backend)
    guess_lambda = guess_lambda_used(real_result['result'])
    return check_value(real_result['lambda_used'], guess_lambda)

In [72]:
def get_combinations_two_etas_without_repeats(input_list_eta):
    """ from a given list of attenuation angles (etas) create a 
        list of all combinatorial pairs of possible etas
        without repeats (order does not matter).
        For us it is the same testing first eta 0.1 and second eta 0.2 
        than first eta 0.2 and second eta 0.1
    """
    list_eta = input_list_eta
     # when there is only one element, we add the same element
    if len(list_eta) == 1:
        list_eta.append(list_eta[0])
    # get combinations of two etas without repeats
    return list(itertools.combinations(list_eta, 2))

## Creating our cost function to be minimized (minimizing the error)

In [73]:
def play_one_configuration(configuration, backend, plays=1000):
    """ Play the play_and_guess game for the number of plays for the configuration of input parameters.
        Returns the average success probability for all the plays with the given configuration. 
    """
    success_counts = 0
    for play in range(plays):
        success_counts += play_and_guess_one_case(configuration, backend)
    return success_counts/plays

In [74]:
def cost_function(params):
    """ Computes the cost of running a specific configuration.
        Cost is computed as 1 (perfect probability) - given probability [0,1]
    """
    backend = backend=Aer.get_backend('qasm_simulator')

    configuration = {
        "theta": params[0],
        "phi_rx": params[1],
        "phi_ry": params[2],
        "lambda0": GLOBAL_LAMBDA0,
        "lambda1": GLOBAL_LAMBDA1
    }
    return 1 - play_one_configuration(configuration, backend, plays=100)

## Parameter Optimization for the best configuration

In [75]:
def compute_best_configuration( optimizer_algorithms=['CRS'], 
                                list_optimizer_iterations=[1000], 
                                list_step_size=[pi/10], 
                                list_iterations_per_algorithm=[1]):
    """ Find out the best configuration with a global pair of lambdas (channels) trying out a list of specified optimizat algorithm """
    # Create the initial parameters and set the bounds
    initial_params = [0, 0, 0]
    variable_bounds = [
                        (0, pi/2), # theta
                        (0, 2*pi), # rx
                        (0, 2*pi)  # ry
                        ]
    best_probability = 0
    best_configuration = []
    best_optimizer_algorithm = ""

    for optimizer_algorithm, optimizer_iterations, step_size, iterations in zip(optimizer_algorithms, 
                                                                    list_optimizer_iterations, 
                                                                    list_step_size, 
                                                                    list_iterations_per_algorithm):
        print("Analyzing Optimizer Algorithm: ", optimizer_algorithm)
        if optimizer_algorithm == 'GSLS':
            # optimizer = GSLS()
            optimizer = GSLS(maxiter=optimizer_iterations, max_eval=optimizer_iterations, initial_step_size=step_size)
        if optimizer_algorithm == 'L_BFGS_B':
            # optimizer = L_BFGS_B()
            optimizer = L_BFGS_B(maxfun=optimizer_iterations, maxiter=optimizer_iterations, epsilon=step_size)
        if optimizer_algorithm == 'SLSQP':
            # optimizer = SLSQP()
            optimizer = SLSQP(maxiter=optimizer_iterations,  eps=step_size)
        if optimizer_algorithm == 'TNC':
            # optimizer = TNC()
            optimizer = TNC(maxiter=optimizer_iterations, eps=step_size)
        if optimizer_algorithm == 'BOBYQA':
            optimizer = BOBYQA(maxiter=optimizer_iterations)
        if optimizer_algorithm == 'IMFIL':
            optimizer = BOBYQA(maxiter=optimizer_iterations)
        if optimizer_algorithm == 'SNOBFIT':
            optimizer = BOBYQA(maxiter=optimizer_iterations)
        if optimizer_algorithm == 'CRS':
            optimizer = CRS(max_evals=optimizer_iterations)
        if optimizer_algorithm == 'DIRECT_L':
            optimizer = DIRECT_L(max_evals=optimizer_iterations)
        if optimizer_algorithm == 'DIRECT_L_RAND':
            optimizer = DIRECT_L_RAND(max_evals=optimizer_iterations)
        if optimizer_algorithm == 'ESCH':
            optimizer = ESCH(max_evals=optimizer_iterations)
        if optimizer_algorithm == 'ISRES':
            optimizer = ISRES(max_evals=optimizer_iterations)

        
        for i in range(iterations):
            ret = optimizer.optimize(num_vars=len(initial_params), 
                                    objective_function=cost_function, 
                                    variable_bounds=variable_bounds,
                                    initial_point=initial_params)
            print("Best Average Probability:", 1 - ret[1])
            if (1-ret[1]) > best_probability:
                best_probability = 1-ret[1]
                best_configuration = ret[0]
                best_optimizer_algorithm = optimizer_algorithm

    # Print results
    print("Final Best Optimizer Algorithm: ", best_optimizer_algorithm)
    print("Final Best Average Probability:", best_probability)
    print("Parameters Found: " + u"\u03B8" + " = " + str(int(math.degrees(best_configuration[0]))) + u"\u00B0" + 
                            ", " + u"\u03D5" + "rx = " + str(int(math.degrees(best_configuration[1]))) + u"\u00B0" + 
                            ", " + u"\u03D5" + "ry = " + str(int(math.degrees(best_configuration[2]))) + u"\u00B0" +
                            ", " + u"\u03B7" + u"\u2080" + " = " + str(GLOBAL_LAMBDA0) + 
                            ", " + u"\u03B7" + u"\u2081" + " = " + str(GLOBAL_LAMBDA1))

    return {
        "best_optimizer_algorithm": best_optimizer_algorithm,
        "best_probability": best_probability,
        "best_configuration": best_configuration
    }


## Find out the best configuration ($\theta, phase, rx, ry$) and success probability for each pair of $\lambda$ attenuation levels from 0 to 1

In [59]:
# create the channels to be used
# from 0 to 1 in steps of 0.1, and adding the 1 as the last value
etas = np.append(np.arange(0, pi/2, pi/2/20), pi/2)

optimizer_algorithms = ['CRS']
list_optimizer_iterations=[1000]
list_step_size=[pi/10]
list_iterations_per_algorithm=[1]
# optimizer_algorithms = ['CRS', 'DIRECT_L', 'DIRECT_L_RAND', 'ESCH', 'ISRES']
# list_optimizer_iterations=[1000, 1000, 1000, 1000, 1000]
# list_step_size=[pi/10, pi/10, pi/10, pi/10, pi/10]
# list_iterations_per_algorithm=[1, 1, 1, 1, 1]
# optimizer_algorithms = ['GSLS', 'L_BFGS_B', 'SLSQP', 'TNC']
# list_optimizer_iterations=[10000, 1000, 100, 100]
# list_step_size=[pi/10, pi/10, pi/10, pi/10]
# list_iterations_per_algorithm=[1, 1, 1, 1]
two_pair_etas = get_combinations_two_etas_without_repeats(etas)
probabilities = []
configurations = []
best_algorithm = []

total_eta_pairs = len(two_pair_etas)

if clones > 1:
    eta_pair_ini = np.ceil(clon*total_eta_pairs/clones)
    eta_pair_fin = min(np.ceil((clon+1)*total_eta_pairs/clones)-1, total_eta_pairs-1)
else:
    eta_pair_ini = 0
    eta_pair_fin = total_eta_pairs-1

program_start_time = time.time()
print("Starting the execution")

for eta_pair in two_pair_etas:
    start_time = time.time()
    prob = 0
    config = np.array([0.0, 0.0, 0.0])
    if eta_pair_ini > two_pair_etas.index(eta_pair):
        probabilities.append(prob)
        configurations.append(config)
        best_algorithm.append('NA')
    elif eta_pair_fin < two_pair_etas.index(eta_pair):
        probabilities.append(prob)
        configurations.append(config)
        best_algorithm.append('NA')
    else:
        GLOBAL_LAMBDA0=eta_pair[0]
        GLOBAL_LAMBDA1=eta_pair[1]
        result = compute_best_configuration(optimizer_algorithms=optimizer_algorithms,
                                        list_optimizer_iterations=list_optimizer_iterations,
                                        list_step_size=list_step_size, 
                                        list_iterations_per_algorithm=list_iterations_per_algorithm)
        probabilities.append(result['best_probability'])
        configurations.append(result['best_configuration'])
        best_algorithm.append(result['best_optimizer_algorithm'])
    end_time = time.time()
    print("total minutes taken this pair of etas: ", int(np.round((end_time - start_time)/60)))
    print("total minutes taken so far: ", int(np.round((end_time - program_start_time)/60)))

end_time = time.time()
print("total minutes of execution time: ", int(np.round((end_time - program_start_time)/60)))
print("All guesses have been calculated")
print(f'Total pair of etas tested: {len(two_pair_etas)}')

result = {
    "lambda_pairs": two_pair_etas,
    "best_algorithm": best_algorithm,
    "probabilities": probabilities,
    "configurations": configurations
}

Starting the execution
Analyzing Optimizer Algorithm:  CRS
Best Average Probability: 0.54
Final Best Optimizer Algorithm:  CRS
Final Best Average Probability: 0.54
Parameters Found: θ = 0°, ϕrx = 0°, ϕry = 0°, η₀ = 0.0, η₁ = 0.07853981633974483
total minutes taken this pair of etas:  0
total minutes taken so far:  0
Analyzing Optimizer Algorithm:  CRS
Best Average Probability: 0.52
Final Best Optimizer Algorithm:  CRS
Final Best Average Probability: 0.52
Parameters Found: θ = 0°, ϕrx = 0°, ϕry = 0°, η₀ = 0.0, η₁ = 0.15707963267948966
total minutes taken this pair of etas:  0
total minutes taken so far:  0
Analyzing Optimizer Algorithm:  CRS
Best Average Probability: 0.43999999999999995
Final Best Optimizer Algorithm:  CRS
Final Best Average Probability: 0.43999999999999995
Parameters Found: θ = 0°, ϕrx = 0°, ϕry = 0°, η₀ = 0.0, η₁ = 0.23561944901923448
total minutes taken this pair of etas:  0
total minutes taken so far:  0
Analyzing Optimizer Algorithm:  CRS
Best Average Probability: 

Best Average Probability: 0.58
Final Best Optimizer Algorithm:  CRS
Final Best Average Probability: 0.58
Parameters Found: θ = 0°, ϕrx = 0°, ϕry = 0°, η₀ = 0.07853981633974483, η₁ = 0.7068583470577035
total minutes taken this pair of etas:  0
total minutes taken so far:  0
Analyzing Optimizer Algorithm:  CRS
Best Average Probability: 0.53
Final Best Optimizer Algorithm:  CRS
Final Best Average Probability: 0.53
Parameters Found: θ = 0°, ϕrx = 0°, ϕry = 0°, η₀ = 0.07853981633974483, η₁ = 0.7853981633974483
total minutes taken this pair of etas:  0
total minutes taken so far:  0
Analyzing Optimizer Algorithm:  CRS
Best Average Probability: 0.53
Final Best Optimizer Algorithm:  CRS
Final Best Average Probability: 0.53
Parameters Found: θ = 0°, ϕrx = 0°, ϕry = 0°, η₀ = 0.07853981633974483, η₁ = 0.8639379797371931
total minutes taken this pair of etas:  0
total minutes taken so far:  0
Analyzing Optimizer Algorithm:  CRS
Best Average Probability: 0.42999999999999994
Final Best Optimizer Alg

Best Average Probability: 0.45999999999999996
Final Best Optimizer Algorithm:  CRS
Final Best Average Probability: 0.45999999999999996
Parameters Found: θ = 0°, ϕrx = 0°, ϕry = 0°, η₀ = 0.15707963267948966, η₁ = 1.413716694115407
total minutes taken this pair of etas:  0
total minutes taken so far:  0
Analyzing Optimizer Algorithm:  CRS
Best Average Probability: 0.5
Final Best Optimizer Algorithm:  CRS
Final Best Average Probability: 0.5
Parameters Found: θ = 0°, ϕrx = 0°, ϕry = 0°, η₀ = 0.15707963267948966, η₁ = 1.4922565104551517
total minutes taken this pair of etas:  0
total minutes taken so far:  0
Analyzing Optimizer Algorithm:  CRS
Best Average Probability: 0.41999999999999993
Final Best Optimizer Algorithm:  CRS
Final Best Average Probability: 0.41999999999999993
Parameters Found: θ = 0°, ϕrx = 0°, ϕry = 0°, η₀ = 0.15707963267948966, η₁ = 1.5707963267948966
total minutes taken this pair of etas:  0
total minutes taken so far:  0
Analyzing Optimizer Algorithm:  CRS
Best Average 

Best Average Probability: 0.5
Final Best Optimizer Algorithm:  CRS
Final Best Average Probability: 0.5
Parameters Found: θ = 0°, ϕrx = 0°, ϕry = 0°, η₀ = 0.3141592653589793, η₁ = 0.8639379797371931
total minutes taken this pair of etas:  0
total minutes taken so far:  1
Analyzing Optimizer Algorithm:  CRS
Best Average Probability: 0.4
Final Best Optimizer Algorithm:  CRS
Final Best Average Probability: 0.4
Parameters Found: θ = 0°, ϕrx = 0°, ϕry = 0°, η₀ = 0.3141592653589793, η₁ = 0.9424777960769379
total minutes taken this pair of etas:  0
total minutes taken so far:  1
Analyzing Optimizer Algorithm:  CRS
Best Average Probability: 0.55
Final Best Optimizer Algorithm:  CRS
Final Best Average Probability: 0.55
Parameters Found: θ = 0°, ϕrx = 0°, ϕry = 0°, η₀ = 0.3141592653589793, η₁ = 1.0210176124166828
total minutes taken this pair of etas:  0
total minutes taken so far:  1
Analyzing Optimizer Algorithm:  CRS
Best Average Probability: 0.5
Final Best Optimizer Algorithm:  CRS
Final Best

Best Average Probability: 0.5
Final Best Optimizer Algorithm:  CRS
Final Best Average Probability: 0.5
Parameters Found: θ = 0°, ϕrx = 0°, ϕry = 0°, η₀ = 0.47123889803846897, η₁ = 0.6283185307179586
total minutes taken this pair of etas:  0
total minutes taken so far:  1
Analyzing Optimizer Algorithm:  CRS
Best Average Probability: 0.5
Final Best Optimizer Algorithm:  CRS
Final Best Average Probability: 0.5
Parameters Found: θ = 0°, ϕrx = 0°, ϕry = 0°, η₀ = 0.47123889803846897, η₁ = 0.7068583470577035
total minutes taken this pair of etas:  0
total minutes taken so far:  1
Analyzing Optimizer Algorithm:  CRS
Best Average Probability: 0.56
Final Best Optimizer Algorithm:  CRS
Final Best Average Probability: 0.56
Parameters Found: θ = 0°, ϕrx = 0°, ϕry = 0°, η₀ = 0.47123889803846897, η₁ = 0.7853981633974483
total minutes taken this pair of etas:  0
total minutes taken so far:  1
Analyzing Optimizer Algorithm:  CRS
Best Average Probability: 0.49
Final Best Optimizer Algorithm:  CRS
Final 

Best Average Probability: 0.48
Final Best Optimizer Algorithm:  CRS
Final Best Average Probability: 0.48
Parameters Found: θ = 0°, ϕrx = 0°, ϕry = 0°, η₀ = 0.6283185307179586, η₁ = 0.7853981633974483
total minutes taken this pair of etas:  0
total minutes taken so far:  1
Analyzing Optimizer Algorithm:  CRS
Best Average Probability: 0.57
Final Best Optimizer Algorithm:  CRS
Final Best Average Probability: 0.57
Parameters Found: θ = 0°, ϕrx = 0°, ϕry = 0°, η₀ = 0.6283185307179586, η₁ = 0.8639379797371931
total minutes taken this pair of etas:  0
total minutes taken so far:  1
Analyzing Optimizer Algorithm:  CRS
Best Average Probability: 0.49
Final Best Optimizer Algorithm:  CRS
Final Best Average Probability: 0.49
Parameters Found: θ = 0°, ϕrx = 0°, ϕry = 0°, η₀ = 0.6283185307179586, η₁ = 0.9424777960769379
total minutes taken this pair of etas:  0
total minutes taken so far:  1
Analyzing Optimizer Algorithm:  CRS
Best Average Probability: 0.47
Final Best Optimizer Algorithm:  CRS
Final

Best Average Probability: 0.5
Final Best Optimizer Algorithm:  CRS
Final Best Average Probability: 0.5
Parameters Found: θ = 0°, ϕrx = 0°, ϕry = 0°, η₀ = 0.7853981633974483, η₁ = 1.2566370614359172
total minutes taken this pair of etas:  0
total minutes taken so far:  1
Analyzing Optimizer Algorithm:  CRS
Best Average Probability: 0.5
Final Best Optimizer Algorithm:  CRS
Final Best Average Probability: 0.5
Parameters Found: θ = 0°, ϕrx = 0°, ϕry = 0°, η₀ = 0.7853981633974483, η₁ = 1.335176877775662
total minutes taken this pair of etas:  0
total minutes taken so far:  1
Analyzing Optimizer Algorithm:  CRS
Best Average Probability: 0.43999999999999995
Final Best Optimizer Algorithm:  CRS
Final Best Average Probability: 0.43999999999999995
Parameters Found: θ = 0°, ϕrx = 0°, ϕry = 0°, η₀ = 0.7853981633974483, η₁ = 1.413716694115407
total minutes taken this pair of etas:  0
total minutes taken so far:  1
Analyzing Optimizer Algorithm:  CRS
Best Average Probability: 0.58
Final Best Optimiz

Best Average Probability: 0.47
Final Best Optimizer Algorithm:  CRS
Final Best Average Probability: 0.47
Parameters Found: θ = 0°, ϕrx = 0°, ϕry = 0°, η₀ = 1.0210176124166828, η₁ = 1.4922565104551517
total minutes taken this pair of etas:  0
total minutes taken so far:  2
Analyzing Optimizer Algorithm:  CRS
Best Average Probability: 0.53
Final Best Optimizer Algorithm:  CRS
Final Best Average Probability: 0.53
Parameters Found: θ = 0°, ϕrx = 0°, ϕry = 0°, η₀ = 1.0210176124166828, η₁ = 1.5707963267948966
total minutes taken this pair of etas:  0
total minutes taken so far:  2
Analyzing Optimizer Algorithm:  CRS
Best Average Probability: 0.49
Final Best Optimizer Algorithm:  CRS
Final Best Average Probability: 0.49
Parameters Found: θ = 0°, ϕrx = 0°, ϕry = 0°, η₀ = 1.0995574287564276, η₁ = 1.1780972450961724
total minutes taken this pair of etas:  0
total minutes taken so far:  2
Analyzing Optimizer Algorithm:  CRS
Best Average Probability: 0.51
Final Best Optimizer Algorithm:  CRS
Final

In [78]:
""" Save results to disk """
save_results_to_disk(result, name=file1)

In [18]:
etas = np.append(np.arange(0, pi/2, pi/2/20), pi/2)

optimizer_algorithms = ['DIRECT_L']
list_optimizer_iterations=[1000]
list_step_size=[pi/10]
list_iterations_per_algorithm=[1]

two_pair_etas = get_combinations_two_etas_without_repeats(etas)
probabilities = []
configurations = []
best_algorithm = []

total_eta_pairs = len(two_pair_etas)

if clones > 1:
    eta_pair_ini = np.ceil(clon*total_eta_pairs/clones)
    eta_pair_fin = min(np.ceil((clon+1)*total_eta_pairs/clones)-1, total_eta_pairs-1)
else:
    eta_pair_ini = 0
    eta_pair_fin = total_eta_pairs-1

program_start_time = time.time()
print("Starting the execution")

for eta_pair in two_pair_etas:
    start_time = time.time()
    prob = 0
    config = np.array([0.0, 0.0, 0.0])
    if eta_pair_ini > two_pair_etas.index(eta_pair):
        probabilities.append(prob)
        configurations.append(config)
        best_algorithm.append('NA')
    elif eta_pair_fin < two_pair_etas.index(eta_pair):
        probabilities.append(prob)
        configurations.append(config)
        best_algorithm.append('NA')
    else:
        GLOBAL_LAMBDA0=eta_pair[0]
        GLOBAL_LAMBDA1=eta_pair[1]
        result = compute_best_configuration(optimizer_algorithms=optimizer_algorithms,
                                        list_optimizer_iterations=list_optimizer_iterations,
                                        list_step_size=list_step_size, 
                                        list_iterations_per_algorithm=list_iterations_per_algorithm)
        probabilities.append(result['best_probability'])
        configurations.append(result['best_configuration'])
        best_algorithm.append(result['best_optimizer_algorithm'])
    end_time = time.time()
    print("total minutes taken this pair of etas: ", int(np.round((end_time - start_time)/60)))
    print("total minutes taken so far: ", int(np.round((end_time - program_start_time)/60)))

end_time = time.time()
print("total minutes of execution time: ", int(np.round((end_time - program_start_time)/60)))
print("All guesses have been calculated")
print(f'Total pair of etas tested: {len(two_pair_etas)}')

result = {
    "lambda_pairs": two_pair_etas,
    "best_algorithm": best_algorithm,
    "probabilities": probabilities,
    "configurations": configurations
}

Starting the execution
Analyzing Optimizer Algorithm:  DIRECT_L
Best Average Probability: 0.63
Final Best Optimizer Algorithm:  DIRECT_L
Final Best Average Probability: 0.63
Parameters Found: θ = 84°, Phase = 60°, ϕrx = 299°, ϕry = 339°, λ₀ = 0.0, λ₁ = 0.07853981633974483
total minutes taken this pair of etas:  19
total minutes taken so far:  19
Analyzing Optimizer Algorithm:  DIRECT_L
Best Average Probability: 0.63
Final Best Optimizer Algorithm:  DIRECT_L
Final Best Average Probability: 0.63
Parameters Found: θ = 45°, Phase = 180°, ϕrx = 55°, ϕry = 180°, λ₀ = 0.0, λ₁ = 0.15707963267948966
total minutes taken this pair of etas:  19
total minutes taken so far:  38
Analyzing Optimizer Algorithm:  DIRECT_L
Best Average Probability: 0.7
Final Best Optimizer Algorithm:  DIRECT_L
Final Best Average Probability: 0.7
Parameters Found: θ = 74°, Phase = 220°, ϕrx = 20°, ϕry = 180°, λ₀ = 0.0, λ₁ = 0.23561944901923448
total minutes taken this pair of etas:  18
total minutes taken so far:  56
Anal

In [19]:
save_results_to_disk(result, name=file2)

In [20]:
# create the channels to be used
# from 0 to 1 in steps of 0.1, and adding the 1 as the last value
etas = np.append(np.arange(0, pi/2, pi/2/20), pi/2)

optimizer_algorithms = ['DIRECT_L_RAND']
list_optimizer_iterations=[1000]
list_step_size=[pi/10]
list_iterations_per_algorithm=[1]

two_pair_etas = get_combinations_two_etas_without_repeats(etas)
probabilities = []
configurations = []
best_algorithm = []

total_eta_pairs = len(two_pair_etas)

if clones > 1:
    eta_pair_ini = np.ceil(clon*total_eta_pairs/clones)
    eta_pair_fin = min(np.ceil((clon+1)*total_eta_pairs/clones)-1, total_eta_pairs-1)
else:
    eta_pair_ini = 0
    eta_pair_fin = total_eta_pairs-1

program_start_time = time.time()
print("Starting the execution")

for eta_pair in two_pair_etas:
    start_time = time.time()
    prob = 0
    config = np.array([0.0, 0.0, 0.0])
    if eta_pair_ini > two_pair_etas.index(eta_pair):
        probabilities.append(prob)
        configurations.append(config)
        best_algorithm.append('NA')
    elif eta_pair_fin < two_pair_etas.index(eta_pair):
        probabilities.append(prob)
        configurations.append(config)
        best_algorithm.append('NA')
    else:
        GLOBAL_LAMBDA0=eta_pair[0]
        GLOBAL_LAMBDA1=eta_pair[1]
        result = compute_best_configuration(optimizer_algorithms=optimizer_algorithms,
                                        list_optimizer_iterations=list_optimizer_iterations,
                                        list_step_size=list_step_size, 
                                        list_iterations_per_algorithm=list_iterations_per_algorithm)
        probabilities.append(result['best_probability'])
        configurations.append(result['best_configuration'])
        best_algorithm.append(result['best_optimizer_algorithm'])
    end_time = time.time()
    print("total minutes taken this pair of etas: ", int(np.round((end_time - start_time)/60)))
    print("total minutes taken so far: ", int(np.round((end_time - program_start_time)/60)))

end_time = time.time()
print("total minutes of execution time: ", int(np.round((end_time - program_start_time)/60)))
print("All guesses have been calculated")
print(f'Total pair of etas tested: {len(two_pair_etas)}')

result = {
    "lambda_pairs": two_pair_etas,
    "best_algorithm": best_algorithm,
    "probabilities": probabilities,
    "configurations": configurations
}

Starting the execution
Analyzing Optimizer Algorithm:  DIRECT_L_RAND
Best Average Probability: 0.66
Final Best Optimizer Algorithm:  DIRECT_L_RAND
Final Best Average Probability: 0.66
Parameters Found: θ = 45°, Phase = 59°, ϕrx = 299°, ϕry = 339°, λ₀ = 0.0, λ₁ = 0.07853981633974483
total minutes taken this pair of etas:  17
total minutes taken so far:  17
Analyzing Optimizer Algorithm:  DIRECT_L_RAND
Best Average Probability: 0.66
Final Best Optimizer Algorithm:  DIRECT_L_RAND
Final Best Average Probability: 0.66
Parameters Found: θ = 15°, Phase = 60°, ϕrx = 299°, ϕry = 299°, λ₀ = 0.0, λ₁ = 0.15707963267948966
total minutes taken this pair of etas:  17
total minutes taken so far:  35
Analyzing Optimizer Algorithm:  DIRECT_L_RAND
Best Average Probability: 0.66
Final Best Optimizer Algorithm:  DIRECT_L_RAND
Final Best Average Probability: 0.66
Parameters Found: θ = 84°, Phase = 193°, ϕrx = 179°, ϕry = 339°, λ₀ = 0.0, λ₁ = 0.23561944901923448
total minutes taken this pair of etas:  17
tot

In [21]:
save_results_to_disk(result, name=file3)

In [22]:
# create the channels to be used
# from 0 to 1 in steps of 0.1, and adding the 1 as the last value
etas = np.append(np.arange(0, pi/2, pi/2/20), pi/2)

optimizer_algorithms = ['ESCH']
list_optimizer_iterations=[1000]
list_step_size=[pi/10]
list_iterations_per_algorithm=[1]

two_pair_etas = get_combinations_two_etas_without_repeats(etas)
probabilities = []
configurations = []
best_algorithm = []

total_eta_pairs = len(two_pair_etas)

if clones > 1:
    eta_pair_ini = np.ceil(clon*total_eta_pairs/clones)
    eta_pair_fin = min(np.ceil((clon+1)*total_eta_pairs/clones)-1, total_eta_pairs-1)
else:
    eta_pair_ini = 0
    eta_pair_fin = total_eta_pairs-1

program_start_time = time.time()
print("Starting the execution")

for eta_pair in two_pair_etas:
    start_time = time.time()
    prob = 0
    config = np.array([0.0, 0.0, 0.0])
    if eta_pair_ini > two_pair_etas.index(eta_pair):
        probabilities.append(prob)
        configurations.append(config)
        best_algorithm.append('NA')
    elif eta_pair_fin < two_pair_etas.index(eta_pair):
        probabilities.append(prob)
        configurations.append(config)
        best_algorithm.append('NA')
    else:
        GLOBAL_LAMBDA0=eta_pair[0]
        GLOBAL_LAMBDA1=eta_pair[1]
        result = compute_best_configuration(optimizer_algorithms=optimizer_algorithms,
                                        list_optimizer_iterations=list_optimizer_iterations,
                                        list_step_size=list_step_size, 
                                        list_iterations_per_algorithm=list_iterations_per_algorithm)
        probabilities.append(result['best_probability'])
        configurations.append(result['best_configuration'])
        best_algorithm.append(result['best_optimizer_algorithm'])
    end_time = time.time()
    print("total minutes taken this pair of etas: ", int(np.round((end_time - start_time)/60)))
    print("total minutes taken so far: ", int(np.round((end_time - program_start_time)/60)))

end_time = time.time()
print("total minutes of execution time: ", int(np.round((end_time - program_start_time)/60)))
print("All guesses have been calculated")
print(f'Total pair of etas tested: {len(two_pair_etas)}')

result = {
    "lambda_pairs": two_pair_etas,
    "best_algorithm": best_algorithm,
    "probabilities": probabilities,
    "configurations": configurations
}

Starting the execution
Analyzing Optimizer Algorithm:  ESCH
Best Average Probability: 0.64
Final Best Optimizer Algorithm:  ESCH
Final Best Average Probability: 0.64
Parameters Found: θ = 54°, Phase = 13°, ϕrx = 53°, ϕry = 230°, λ₀ = 0.0, λ₁ = 0.07853981633974483
total minutes taken this pair of etas:  18
total minutes taken so far:  18
Analyzing Optimizer Algorithm:  ESCH
Best Average Probability: 0.67
Final Best Optimizer Algorithm:  ESCH
Final Best Average Probability: 0.67
Parameters Found: θ = 11°, Phase = 186°, ϕrx = 260°, ϕry = 189°, λ₀ = 0.0, λ₁ = 0.15707963267948966
total minutes taken this pair of etas:  17
total minutes taken so far:  35
Analyzing Optimizer Algorithm:  ESCH
Best Average Probability: 0.63
Final Best Optimizer Algorithm:  ESCH
Final Best Average Probability: 0.63
Parameters Found: θ = 39°, Phase = 230°, ϕrx = 185°, ϕry = 8°, λ₀ = 0.0, λ₁ = 0.23561944901923448
total minutes taken this pair of etas:  17
total minutes taken so far:  52
Analyzing Optimizer Algorit

In [23]:
save_results_to_disk(result, name=file4)

In [24]:
# create the channels to be used
# from 0 to 1 in steps of 0.1, and adding the 1 as the last value
etas = np.append(np.arange(0, pi/2, pi/2/20), pi/2)

optimizer_algorithms = ['ISRES']
list_optimizer_iterations=[1000]
list_step_size=[pi/10]
list_iterations_per_algorithm=[1]

two_pair_etas = get_combinations_two_etas_without_repeats(etas)
probabilities = []
configurations = []
best_algorithm = []

total_eta_pairs = len(two_pair_etas)

if clones > 1:
    eta_pair_ini = np.ceil(clon*total_eta_pairs/clones)
    eta_pair_fin = min(np.ceil((clon+1)*total_eta_pairs/clones)-1, total_eta_pairs-1)
else:
    eta_pair_ini = 0
    eta_pair_fin = total_eta_pairs-1

program_start_time = time.time()
print("Starting the execution")

for eta_pair in two_pair_etas:
    start_time = time.time()
    prob = 0
    config = np.array([0.0, 0.0, 0.0])
    if eta_pair_ini > two_pair_etas.index(eta_pair):
        probabilities.append(prob)
        configurations.append(config)
        best_algorithm.append('NA')
    elif eta_pair_fin < two_pair_etas.index(eta_pair):
        probabilities.append(prob)
        configurations.append(config)
        best_algorithm.append('NA')
    else:
        GLOBAL_LAMBDA0=eta_pair[0]
        GLOBAL_LAMBDA1=eta_pair[1]
        result = compute_best_configuration(optimizer_algorithms=optimizer_algorithms,
                                        list_optimizer_iterations=list_optimizer_iterations,
                                        list_step_size=list_step_size, 
                                        list_iterations_per_algorithm=list_iterations_per_algorithm)
        probabilities.append(result['best_probability'])
        configurations.append(result['best_configuration'])
        best_algorithm.append(result['best_optimizer_algorithm'])
    end_time = time.time()
    print("total minutes taken this pair of etas: ", int(np.round((end_time - start_time)/60)))
    print("total minutes taken so far: ", int(np.round((end_time - program_start_time)/60)))

end_time = time.time()
print("total minutes of execution time: ", int(np.round((end_time - program_start_time)/60)))
print("All guesses have been calculated")
print(f'Total pair of etas tested: {len(two_pair_etas)}')

result = {
    "lambda_pairs": two_pair_etas,
    "best_algorithm": best_algorithm,
    "probabilities": probabilities,
    "configurations": configurations
}

Starting the execution
Analyzing Optimizer Algorithm:  ISRES
Best Average Probability: 0.66
Final Best Optimizer Algorithm:  ISRES
Final Best Average Probability: 0.66
Parameters Found: θ = 43°, Phase = 160°, ϕrx = 61°, ϕry = 228°, λ₀ = 0.0, λ₁ = 0.07853981633974483
total minutes taken this pair of etas:  17
total minutes taken so far:  17
Analyzing Optimizer Algorithm:  ISRES
Best Average Probability: 0.68
Final Best Optimizer Algorithm:  ISRES
Final Best Average Probability: 0.68
Parameters Found: θ = 39°, Phase = 193°, ϕrx = 73°, ϕry = 258°, λ₀ = 0.0, λ₁ = 0.15707963267948966
total minutes taken this pair of etas:  17
total minutes taken so far:  35
Analyzing Optimizer Algorithm:  ISRES
Best Average Probability: 0.65
Final Best Optimizer Algorithm:  ISRES
Final Best Average Probability: 0.65
Parameters Found: θ = 58°, Phase = 355°, ϕrx = 17°, ϕry = 241°, λ₀ = 0.0, λ₁ = 0.23561944901923448
total minutes taken this pair of etas:  17
total minutes taken so far:  52
Analyzing Optimizer 

In [25]:
save_results_to_disk(result, name=file5)